In [19]:
import numpy as np

# Unit Tests

## Testing Principles
Testing is the process by which you exercise your code to determine if it performs as expected. This means that a test has two parts:
- An invocation of your code that exercises particular your code in particular ways.
- An evaluation of the result and/or state of your code after execution completes

Testing is *not* a standalone process. Typically, you are developing tests while you develop code. A common practice is called **test-driven development**. You write the tests *before* you implement the code. At a minimum, you will be modifying code as you find bugs.

## Applying the Testing Principles

### Entropy of a set of probabilities
$$
H = -\sum_i p_i \log(p_i)
$$

In [17]:
# Code for computing Entropy
def entropy(ps):
    items = ps * np.log(ps)
    return -np.sum(items)

Some kinds of tests
- "Simple test" - run it for a case where the result is known
- "Smoke test" - run it and see if it breaks
- "Edge test" - run it for a case where it might break
- "Exploratory test" - run it for cases where we don't know the result but we can bound what the result should be

In [28]:
# Another simple test
# Simple test
size = 2
prob = 1.0/size
ps = np.repeat(prob , size)
if entropy(ps) != -np.log(prob):
    print ("Got a bad result.")

It would be good to do a lot of tests. But it's cumbersome to organize them and check the results. We need some infrastructure support.

## Unittest Infrastructure

Now that we know that certain arguments are invalid, we should test for this. Let's start with a little bit of infrastructure, the `unittest` framework provided by python.

Using this infrastructure, requires the following:
1. import the unittest module
1. define a class that inherits from unittest.TestCase
1. write methods that run the code to be tested and check the outcomes

The last item is done by using `assert` methods. For example, `self.assertEqual` takes two arguments. If these are objects for which `==` returns `True`, then the test passes. Otherwise, the test fails.

In [14]:
import unittest

# Define a class in which the tests will run
class UnitTests(unittest.TestCase):

    # Each method in the class to execute a test
    def test_upper(self):
        self.assertEqual(1, 1)

    def test_isupper(self):
        self.assertEqual(1, 2)


In [10]:
# Running unittests inside Jupyter requires some special code.
# This code is encapsulated inthe function below. When you create
# files containing unittests, it will look simpler.
def test(test_class=UnitTests):
    # Convenience function to run tests.
    # test_class is the class containing the tests.
    suite = unittest.TestLoader().loadTestsFromModule(test_class())
    unittest.TextTestRunner().run(suite)

In [15]:
# The function test runs the class UnitTests.
test()

.F
FAIL: test_upper (__main__.UnitTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-e37d7d47f3cc>", line 9, in test_upper
    self.assertEqual(1, 2)
AssertionError: 1 != 2

----------------------------------------------------------------------
Ran 2 tests in 0.006s

FAILED (failures=1)


As expected, the first test passes, but the second test fails.

Below, we test the `entropy` function using the unittest infrastructure.

In [33]:
import unittest

# Define a class in which the tests will run
class EntropyTest(unittest.TestCase):

    def testSimple(self):
        self.assertEqual(entropy(1.0), 0.0)
        
test(test_class=EntropyTest)


.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


Although there's some setup, the tests are much easier than what we did before
```
ps = 1.0  # Should have an entropy of 0
if entropy(ps) != 0:
    print ("Got a bad result.")
```

Now we can add LOTS of tests.

In [34]:
import unittest

# Define a class in which the tests will run
class EntropyTest(unittest.TestCase):

    def testSimple(self):
        self.assertEqual(entropy(1.0), 0.0)
        
    def _testEqualProbability(self, size):
        prob = 1.0/size
        ps = np.repeat(prob , size)
        self.assertEqual(entropy(ps), -np.log(prob))
        
    def testEqualProbability(self):
        self._testEqualProbability(2)
        self._testEqualProbability(20)
        self._testEqualProbability(200)
        
test(test_class=EntropyTest)


F.
FAIL: testEqualProbability (__main__.EntropyTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-34-9fca0ad64d47>", line 17, in testEqualProbability
    self._testEqualProbability(200)
  File "<ipython-input-34-9fca0ad64d47>", line 12, in _testEqualProbability
    self.assertEqual(entropy(ps), -np.log(prob))
AssertionError: 5.2983173665480372 != 5.2983173665480363

----------------------------------------------------------------------
Ran 2 tests in 0.011s

FAILED (failures=1)


Why did this test fail? How do we deal with this?

In [36]:
import unittest

# Define a class in which the tests will run
class EntropyTest(unittest.TestCase):

    def testSimple(self):
        self.assertEqual(entropy(1.0), 0.0)
        
    def _testEqualProbability(self, size):
        prob = 1.0/size
        ps = np.repeat(prob , size)
        self.assertTrue(np.isclose(entropy(ps), -np.log(prob)))
        
    def testEqualProbability(self):
        self._testEqualProbability(2)
        self._testEqualProbability(20)
        self._testEqualProbability(200)
        
test(test_class=EntropyTest)

..
----------------------------------------------------------------------
Ran 2 tests in 0.009s

OK


## Testing Exceptions

## Test Files
- Show code by creating test_prime.py
- Using nose vs. python to run tests

## Exercise
- Debug prime.py
- Create a test file test_prime.py that contains unit tests

## SAVE

In [ ]:
ps = "0.1, 0.9"
entropy(ps)

In [ ]:
# Code for computing Entropy
def entropy(ps):
    try:
        items = ps * np.log(ps)
    except TypeError:
        raise TypeError("Argument must be an array or list of numbers.")
        return np.nan
    return -np.sum(items)